In [98]:
import sqlite3

# Connect to database
conn = sqlite3.connect('db.sqlite3')

In [33]:
# delete all rows from table
conn.execute("DELETE FROM api_component")
conn.commit()

In [ ]:
class AA:
    def __init__(self):
        self.A = 1
        self.B = 2
        self.C = 3
    def __str__(self):
        return f"{self.A} {self.B} {self.C}"
    def __repr__(self):
        return f"{self.A} {self.B}"
    def __call__(self, *args):
        for arg in args:
            if arg == "1":
                print("arg 1")
            elif arg == "2":
                print("arg 2")

In [2]:
import joblib
joblib.load(r"C:\Users\a1mme\OneDrive\Desktop\MO\test_grad\project\core\nodes\saved\data\data_loader_1775997487041.pkl")

array([[ 0.03807591,  0.05068012,  0.06169621, ..., -0.00259226,
         0.01990749, -0.01764613],
       [-0.00188202, -0.04464164, -0.05147406, ..., -0.03949338,
        -0.06833155, -0.09220405],
       [ 0.08529891,  0.05068012,  0.04445121, ..., -0.00259226,
         0.00286131, -0.02593034],
       ...,
       [ 0.04170844,  0.05068012, -0.01590626, ..., -0.01107952,
        -0.04688253,  0.01549073],
       [-0.04547248, -0.04464164,  0.03906215, ...,  0.02655962,
         0.04452873, -0.02593034],
       [-0.04547248, -0.04464164, -0.0730303 , ..., -0.03949338,
        -0.00422151,  0.00306441]], shape=(442, 10))

In [ ]:
import requests
import json
url = "http://127.0.0.1:8000/api/nodes/"
response = requests.get(url)
data = json.loads(response.content.decode())
for node in data:
    del node["created_at"]
    del node["updated_at"]
with open("data.json", "w") as f:
    f.write(json.dumps(data, indent=4))


In [4]:
import json
import sys
import base64
import hashlib
import os
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

os.environ["AESC_KEY"] = "my_secret"

class AinoprjConverter:
    types = {
        "node_id": "int", "node_name": "str", "node_type": "str", 
        "task": "str", "message": "str", "node_data": "str",
        "params": "dict", "x_loc": "int", "y_loc": "int",
        "input_nodes": "list", "output_nodes": "list"
    }

    def __init__(self, in_format, out_format, source_path, destination_path, encrypt):
        self.in_format = in_format
        self.out_format = out_format
        self.source_path = source_path
        self.destination_path = destination_path
        self.encrypt = encrypt
        self.SECRET_KEY = self.get_secret_key()

    def get_secret_key(self):
        """Retrieve the encryption key from an environment variable."""
        key = os.getenv("AESC_KEY")
        if not key:
            raise ValueError("Encryption key not set! Use: export AESC_KEY='your_key'")
        return hashlib.sha256(key.encode()).digest()[:16]  # Derive 16-byte key from SHA-256 hash

    def encrypt_data(self, data):
        if not self.encrypt:
            return data
        cipher = AES.new(self.SECRET_KEY, AES.MODE_CBC, iv=self.SECRET_KEY)  # Using key as IV (for simplicity)
        encrypted_bytes = cipher.encrypt(pad(data.encode(), AES.block_size))
        return base64.b64encode(encrypted_bytes).decode()

    def decrypt_data(self, encrypted_data):
        if not self.encrypt:
            return encrypted_data
        cipher = AES.new(self.SECRET_KEY, AES.MODE_CBC, iv=self.SECRET_KEY)
        decrypted_bytes = unpad(cipher.decrypt(base64.b64decode(encrypted_data)), AES.block_size)
        return decrypted_bytes.decode()

    def json_to_ainoprj(self):
        try:
            with open(self.source_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            ainoprj_data = "AINOPRJ_START\n"
            for node in data:
                ainoprj_data += "\tNODE_START\n"
                for key, value in node.items():
                    type_value = self.types.get(key, "na")
                    ainoprj_data += f"\t\t{key}={value}={type_value}\n"
                ainoprj_data += "\tNODE_END\n"
            ainoprj_data += "AINOPRJ_END"
            
            encrypted_data = self.encrypt_data(ainoprj_data)
            with open(self.destination_path, 'w', encoding='utf-8') as f:
                f.write(encrypted_data)
            
            print("Conversion successful: JSON to AINOPRJ (Encrypted)")
        except Exception as e:
            print(f"Error: {e}")

    def ainoprj_to_json(self):
        self.encrypt = False
        try:
            with open(self.source_path, 'r', encoding='utf-8') as f:
                encrypted_data = f.read()
                if len(encrypted_data.split('\n')) == 1:
                    self.encrypt = True  # If data is a single line, assume it's encrypted
                decrypted_data = self.decrypt_data(encrypted_data)
            
            nodes = []
            for line in decrypted_data.split('\n'):
                line = line.lstrip()
                if line in ["AINOPRJ_START", "AINOPRJ_END"]:
                    continue
                elif line == "NODE_START":
                    current_node = {}
                elif line == "NODE_END":
                    nodes.append(current_node)
                    current_node = None
                else:
                    try:
                        key, rest = line.split('=', 1)
                        value, type_string = rest.rsplit('=', 1)
                        current_node[key] = value if type_string == "str" else eval(value)
                    except ValueError as ve:
                        print(f"Skipping malformed line: {key} | Error: {ve}")
            
            with open(self.destination_path, 'w', encoding='utf-8') as f:
                json.dump(nodes, f, indent=4)
            
            print("Conversion successful: AINOPRJ to JSON (Decrypted)")
        except Exception as e:
            print(f"Error: {e}")

    def convert(self):
        if self.in_format == self.out_format:
            print("Error: Input and output formats must be different.")
            sys.exit(1)
        
        if self.in_format == "json" and self.out_format == "ainoprj":
            self.json_to_ainoprj()
        elif self.in_format == "ainoprj" and self.out_format == "json":
            self.ainoprj_to_json()
        else:
            print("Error: Invalid conversion type.")
            sys.exit(1)

# Example usage
converter = AinoprjConverter("json", "ainoprj", "data.json", "data.ainoprj", True)
converter.convert()
converter = AinoprjConverter("ainoprj", "json", "data.ainoprj", "data.json", True)
converter.convert()


Conversion successful: JSON to AINOPRJ (Encrypted)
Conversion successful: AINOPRJ to JSON (Decrypted)


In [182]:
def char_to_bin(char):
    ascii_num = ord(char)
    bin_number = bin(ascii_num)[2:]
    binary_list = list(bin_number.zfill(8))
    return binary_list
def bin_to_char(bin_str,n):
    output = chr(int(bin_str,2) + n)
    return output


def str_to_bin(string_):
    import numpy as np
    output = np.array(list(map(lambda x: char_to_bin(x), string_))).ravel().tolist()
    output = ''.join(output)
    return output

def bin_to_str(bin_str, n):
    bin_str = [bin_str[i:i+n] for i in range(0, len(bin_str), n)]
    output = ''.join([bin_to_char(i,2**n+1) for i in bin_str])
    return output

bin_to_str(str_to_bin("Man "), 6)

'TWFoIA'

In [1]:
import joblib
joblib.load(r"C:\Users\a1mme\OneDrive\Desktop\MO\test_grad\project\core\nodes\saved\data\data_loader_1494029361921.pkl")

b'\x80\x04\x95\xd3\x00\x00\x00\x00\x00\x00\x00\x8c\x13joblib.numpy_pickle\x94\x8c\x11NumpyArrayWrapper\x94\x93\x94)\x81\x94}\x94(\x8c\x08subclass\x94\x8c\x05numpy\x94\x8c\x07ndarray\x94\x93\x94\x8c\x05shape\x94K\x96K\x04\x86\x94\x8c\x05order\x94\x8c\x01C\x94\x8c\x05dtype\x94h\x06\x8c\x05dtype\x94\x93\x94\x8c\x02f8\x94\x89\x88\x87\x94R\x94(K\x03\x8c\x01<\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00t\x94b\x8c\nallow_mmap\x94\x88\x8c\x1bnumpy_array_alignment_bytes\x94K\x10ub\x01\xffffffff\x14@\x00\x00\x00\x00\x00\x00\x0c@ffffff\xf6?\x9a\x99\x99\x99\x99\x99\xc9?\x9a\x99\x99\x99\x99\x99\x13@\x00\x00\x00\x00\x00\x00\x08@ffffff\xf6?\x9a\x99\x99\x99\x99\x99\xc9?\xcd\xcc\xcc\xcc\xcc\xcc\x12@\x9a\x99\x99\x99\x99\x99\t@\xcd\xcc\xcc\xcc\xcc\xcc\xf4?\x9a\x99\x99\x99\x99\x99\xc9?ffffff\x12@\xcd\xcc\xcc\xcc\xcc\xcc\x08@\x00\x00\x00\x00\x00\x00\xf8?\x9a\x99\x99\x99\x99\x99\xc9?\x00\x00\x00\x00\x00\x00\x14@\xcd\xcc\xcc\xcc\xcc\xcc\x0c@ffffff\xf6?\x9a\x99\x99\x99\x99\x99\xc9?\x9a\x99\x99\x99\x99\x99\x1